<a href="https://colab.research.google.com/github/Rajeevku1/AIAgentSamples/blob/main/ChatusingGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import requests
import gradio as gr
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import userdata
import json
import math


In [12]:
openai_api_key = userdata.get('OPENAI_API_KEY')
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

# Initialize OpenAI clients
if openai_api_key:
    openai_client = OpenAI(api_key=openai_api_key)
else:
    openai_client = None

anthropic_url = "https://api.anthropic.com/v1/"
if anthropic_api_key:
    anthropic_client = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
else:
    anthropic_client = None

In [13]:
def stream_claude(prompt):
    sys_msg_claude = "You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way"
    messages = [
        {"role": "system", "content": sys_msg_claude},
        {"role": "user", "content": prompt}
      ]
    stream = anthropic_client.chat.completions.create(
        model="claude-sonnet-4-5-20250929",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
    return result

In [14]:
def stream_model(prompt, model):
    match model:
      case "GPT":
          result = stream_gpt(prompt, model)
      case "Claude":
          result = stream_claude(prompt)
      case "GPT Reasoning":
          result = stream_gpt(prompt, model)
      case "GPT Internet":
          result = stream_gpt(prompt, model)
      case "GPT Intentbased":
          result = stream_gpt(prompt, model)
      case _:
          raise ValueError("Unknown model")
    return result

In [15]:
def stream_gpt(prompt,model):
    if not openai_client:
        return "Error: OpenAI API key not set"
    mean_confidence = 0
    system_message = "You are a helpful assistant can use internet to answer queries"
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    match model:
        case "GPT Reasoning":
            response = openai_client.chat.completions.create(model="gpt-5-mini", messages=messages, reasoning_effort="minimal")
        case "GPT Internet":
            promptForInternet = ask_with_internet(prompt)
            messages = [{"role": "user", "content": promptForInternet},{"role": "system", "content": system_message}]
            #print(messages)
            response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages, logprobs=True, top_logprobs=1  )
        case  "GPT":
          response = openai_client.chat.completions.create(model="gpt-4", messages=messages, logprobs=True, top_logprobs=1 )
        case  "GPT Intentbased":
            sysmsg_intent = "You are an intent classifier for a credit card CRM. Return Only customer intents and nothing else"
            msg = [{"role": "system", "content": sysmsg_intent}, {"role": "user", "content": prompt}]
            response = openai_client.chat.completions.create(model="gpt-5-mini", messages=msg,temperature=1)
    if model=="GPT Internet" or model=="GPT":
        token_logprob_objects = response.choices[0].logprobs.content
        token_logprobs_values = [item.logprob for item in token_logprob_objects if item.logprob is not None]
        token_probs = [math.exp(lp) for lp in token_logprobs_values]
        if token_probs:
            mean_confidence = sum(token_probs)/len(token_probs)
        else:
            mean_confidence = 0 # Handle case where no logprobs are available
    return response.choices[0].message.content + f"\n\nMean confidence: {mean_confidence}"

In [16]:
def chatWithAIusingGrido():
    # Let's use Markdown
    # Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
    # I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
    # Not a great software engineering practice, but quite common during Jupyter Lab R&D!

    system_message = "You are a helpful assistant that responds in detail"
    message_input = gr.Textbox(label="Your message:", info="Enter a message", lines=7)
    model_Name  = gr.Dropdown(["GPT", "GPT Reasoning","Claude", "GPT Internet", "GPT Intentbased"],label="Select model", value="GPT")
    message_output = gr.Textbox(label="Response:", lines=12)

    view = gr.Interface(
        fn=stream_model,
        title="AI Chat Bot",
        inputs=[message_input,model_Name],
        outputs=[message_output],
        examples=[
            ["Explain the Transformer architecture to a layperson", "GPT" ],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
            ],
        flagging_mode="never"
        )
    view.launch(inbrowser=True, debug =True)

In [17]:
def tavily_search(query):
    url = "https://api.tavily.com/search"
    payload = {
        "api_key": TAVILY_API_KEY,
        "query": query,
        "max_results": 5
    }
    return requests.post(url, json=payload).json()



In [18]:
def ask_with_internet(question):
    # Step 1: Search
    results = tavily_search(question)

    # Step 2: Extract text
    context = "\n".join([r["content"] for r in results["results"]])

    # Step 3: Ask LLM with context
    prompt = f"""
    Use the following web search results to answer the question.

    Search Results:
    {context}

    Question: {question}

    Answer using the search results.
    """
    #print(prompt)
    return prompt


In [ ]:
#prompt = ask_with_internet("who is president of USA")
response = stream_gpt(prompt="who is president of USA", model="GPT")
print(response)

In [ ]:
chatWithAIusingGrido()